In [2]:
import cv2
import os
from tqdm import tqdm
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
RAW_VIDEOS_DIR = '/home/dzigen/Desktop/ITMO/sem1/DLtech/lab2/raw_videos'
FIXED_VIDEOS_DIR = '/home/dzigen/Desktop/ITMO/sem1/DLtech/lab2/fixed_videos'
FRAMES_DIR = '/home/dzigen/Desktop/ITMO/sem1/DLtech/lab2/frames'
VIDEOS_DATASET_PATH = '/home/dzigen/Desktop/ITMO/sem1/DLtech/lab2/videos_dataset.csv'
LABEL_NAMES = os.listdir(RAW_VIDEOS_DIR)

#### Статистика по собранным видео

In [34]:
# Удаляем дубликаты из скачанных роликов
#
#print(LABEL_NAMES)
#DOWNLOADED_LINKS = {l_name: list(set(os.listdir(f"{RAW_VIDEOS_DIR}/{l_name}"))) for l_name in LABEL_NAMES}

#
#notunique_links = set()
#notunique_links.update(set(DOWNLOADED_LINKS['travel']).intersection(set(DOWNLOADED_LINKS['food'])))
#notunique_links.update(set(DOWNLOADED_LINKS['travel']).intersection(set(DOWNLOADED_LINKS['art_music'])))
#notunique_links.update(set(DOWNLOADED_LINKS['travel']).intersection(set(DOWNLOADED_LINKS['history'])))
#notunique_links.update(set(DOWNLOADED_LINKS['food']).intersection(set(DOWNLOADED_LINKS['art_music'])))
#notunique_links.update(set(DOWNLOADED_LINKS['food']).intersection(set(DOWNLOADED_LINKS['history'])))
#notunique_links.update(set(DOWNLOADED_LINKS['art_music']).intersection(set(DOWNLOADED_LINKS['history'])))
#print(notunique_links)

#
#for key in DOWNLOADED_LINKS.keys():
#    DOWNLOADED_LINKS[key] = list(set(DOWNLOADED_LINKS[key]).difference(notunique_links))

['travel', 'history', 'food', 'art_music']
{'kMwMTKh5jfM.mp4', '#NAME?.mp4', 'CUv9lfFaSYI.mp4', '76jpCIZTq2E.mp4'}


In [50]:
video_links_df = pd.read_csv(VIDEOS_DATASET_PATH, sep=';')
DOWNLOADED_LINKS = {l_name: video_links_df[video_links_df['category'] == l_name]['links'].to_list() for l_name in LABEL_NAMES}

In [51]:
video_framelen = {}
for label in DOWNLOADED_LINKS.keys():
    video_framelen.setdefault(label, [])
    for link in DOWNLOADED_LINKS[label]:
        #print(label, link)
        v = cv2.VideoCapture(f'{FIXED_VIDEOS_DIR}/{label}/{link}')
        frames_amount = int(v.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(frames_amount)
        #break
        video_framelen[label].append(frames_amount)

In [ ]:
print("Frames statistics:\n")
for label in DOWNLOADED_LINKS.keys():
    print(f"{label}:")
    print("\tmean: ", round(np.mean(video_framelen[label]),3))
    print("\tmax: ", np.max(video_framelen[label]))
    print("\tmin: ", np.min(video_framelen[label]))

In [ ]:
# Фильтруем скаченные ролики и формируем таблицу

# Примечание:
# * для 'history' длинные ролики - норм;
# * для 'art_music' длинные ролики (>30 мин) - не норм (в качестве видеоряда картинка/гифка);
# * для 'travel' длинных роликов нет;
# * для 'food' длинных роликов нет.
VIDEO_DURATION_LIMIT = {'travel': -1, 'history': -1, 'food': -1, 'art_music': 30}

data = []
for label in DOWNLOADED_LINKS.keys():
    for link in DOWNLOADED_LINKS[label]:
        v = cv2.VideoCapture(f'{RAW_VIDEOS_DIR}/{label}/{link}')
        frames_amount = int(v.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = v.get(cv2.CAP_PROP_FPS)

        minutes_duration_round = int(frames_amount / fps / 60)
        if VIDEO_DURATION_LIMIT[label] != -1 and minutes_duration_round > VIDEO_DURATION_LIMIT[label]:
            continue

        data.append((link, label))

saved_df = pd.DataFrame(data, columns=['links', 'category'])
print(Counter(saved_df['category']))
saved_df.to_csv("videos_dataset.csv", index=False, sep=';')

BEFORE FILTERING

Frames statistics:

travel:
	mean:  24659.433
	max:  235975
	min:  487
history:
	mean:  87708.131
	max:  4280403
	min:  1622
food:
	mean:  25095.721
	max:  135495
	min:  861
art_music:
	mean:  15075.599
	max:  68425
	min:  2410

AFTER FILTERING

Frames statistics:

travel:
	mean:  24659.436
	max:  235975
	min:  487
history:
	mean:  87389.287
	max:  4280403
	min:  0
food:
	mean:  24965.158
	max:  135495
	min:  0
art_music:
	mean:  12942.243
	max:  55139
	min:  2410

#### Извлечение фреймов из изображений

In [53]:
video_links_df = pd.read_csv(VIDEOS_DATASET_PATH, sep=';')

In [54]:
OUTPUT_PATH = '/home/dzigen/Desktop/ITMO/sem1/DLtech/lab2/frames'
if not os.path.exists(OUTPUT_PATH):
	os.mkdir(OUTPUT_PATH)

In [64]:
len(os.listdir("/home/dzigen/Desktop/ITMO/sem1/DLtech/lab2/frames/food"))

18874

In [60]:
FRAME_INTERVAL = {'travel': 500, 'history': 1000,'food': 500,'art_music': 100}
END_OFFSET = 500 
START_OFFSET = 200

# LABEL_NAMES
for name in ['travel', 'art_music']:
    video_label_dir = f'{FIXED_VIDEOS_DIR}/{name}'

    frame_label_dir = f'{FRAMES_DIR}/{name}'
    if not os.path.exists(frame_label_dir):
        os.mkdir(frame_label_dir)

    label_videos = video_links_df[video_links_df['category'] == name]['links'].to_list()
    
    #
    process = tqdm(label_videos)
    for video_file in process:
        vidcap = cv2.VideoCapture(f"{video_label_dir}/{video_file}")
        frames_amount = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

        if frames_amount == 0:
            print(f"{name} | {video_file} | {frames_amount} frames ")
            continue

        #
        video_link = video_file.split(".mp4")[0]
        for frame_id in range(START_OFFSET, frames_amount-END_OFFSET, FRAME_INTERVAL[name]):
            vidcap.set(1, frame_id)
            ret, frame = vidcap.read()

            process.set_description_str(f"{name} | {video_file} | {frames_amount} frames | {ret}")

            #
            frame_file = f"{video_link}_frame{frame_id}"
            cv2.imwrite(f"{frame_label_dir}/{frame_file}.png", frame)

travel | LdbJ6ancF4Q.mp4 | 15084 frames | True: 100%|██████████| 337/337 [04:54<00:00,  1.14it/s] 
art_music | lpdRqn6xwiM.mp4 | 4295 frames | True: 100%|██████████| 173/173 [05:13<00:00,  1.81s/it] 
